# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [15]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [16]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [20]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
# Unfortunately, since the only key in js is 'Information', which indicates that 
# that the API is not free, we will use yahoo finance as a fallback.
USE_ALPHA = False
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    # Since there is no 'Adj Close' column in the API response, we will use 'Close' instead.
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
    # print('Adj Close' in df_api.columns)
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

C:\Users\ahana\AppData\Local\Temp\ipykernel_9952\2581625142.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
[*********************100%***********************]  1 of 1 completed


{'missing': [], 'shape': (63, 2), 'na_total': 0}

In [21]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-yfinance_symbol-AAPL_20250820-165211.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [27]:
SCRAPE_URL = 'https://finance.yahoo.com/quote/AAPL/history?p=AAPL'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

Scrape failed, using inline demo table: 503 Server Error: Service Unavailable for url: https://finance.yahoo.com/quote/AAPL/history/?p=AAPL


{'missing': [], 'shape': (1, 2), 'na_total': 0}

The above code cell won't work as I keep getting scraping failed errors. So as a result, I will not be able to complete it. :(

In [28]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data\raw\scrape_site-example_table-markets_20250820-171640.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.

# Sources, Parameters, and Validation
## API

Source: yfinance
Parameters: ticker=AAPL, period=1y, interval=1d, auto_adjust=False
Validation: required columns = [date, open, high, low, close, volume]
## Scrape

URL: https://finance.yahoo.com/quote/AAPL/history?p=AAPL
Scrape failed unfortunately.

data/raw/api_yfinance_AAPL_YYYYMMDD-HHMM.csv

data/raw/scrape_site-example_table-markets_YYYYMMDD-HHMM.csv

## Assumptions & Risks
API schema and column names may change over time; cleaning and validation logic must be updated accordingly.

Scraping depends on the HTML structure (table class and column order); if the webpage changes, selectors and parsing logic must be revised.

.env file is for local use only; only .env.example is included in the repo. Sensitive API keys must not be committed.